In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [70]:
pip install -q langchain langchain-community langchain_huggingface chromadb

Note: you may need to restart the kernel to use updated packages.


In [5]:
## 1. Use langchain RAG¶

In [63]:
import os
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

In [66]:
os.environ["HUGGINGFACEHUB_API_TOKEN"] = "YOUR-API-KEY"  

In [67]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.llms import HuggingFaceHub

# set Korean embedding and llm odel
hf_embeddings = HuggingFaceEmbeddings(model_name="jhgan/ko-sroberta-multitask")

hf_llm = HuggingFaceHub(
    repo_id="skt/kogpt2-base-v2",
    model_kwargs={"task": "text-generation"} ## question-answering tasK X. text-generation
)

In [69]:
import requests
from langchain.schema import Document
from bs4 import BeautifulSoup

# for Wikipedia documents (EN, KO)

# from langchain_community.document_loaders import WikipediaLoader

# By default, English documents (https://en.wikipedia.org))
# def load_Wiki_docs(query):
#     loader = WikipediaLoader(query=query, load_max_docs=1) # need !pip install wikipedia
#     documents = loader.load()
    
#     text_splitter = RecursiveCharacterTextSplitter(
#         chunk_size=1000,
#         chunk_overlap=200
#     )
#     splits = text_splitter.split_documents(documents)
    
#     return splits


# For Korean query, get results from Korean wikipedia website and crawl and parse results
def load_Korean_wiki_docs(topic):
    url = f"https://ko.wikipedia.org/wiki/{topic}"
    
    response = requests.get(url)
    response.raise_for_status()  # raise Exception when error occurs

    # HTML parsing and extract body contents
    soup = BeautifulSoup(response.text, 'html.parser')
    content = soup.find('div', {'class': 'mw-parser-output'})  # find div including body contents 
    
    # Extract contents
    paragraphs = content.find_all('p')
    text = "\n".join([p.get_text() for p in paragraphs])  # concat all context in <p> tags 
 
    # convert to Document object (required for LangChain)
    documents = [Document(page_content=text, metadata={"source": url})] # 제품명 : 설화수, 가격 : 4,000원 이런 식으로 임베딩 하지 않은 애들을 넣는다 

# page_content="구성성분 : 비타민"

    # for 한 화장품 단위 in 긁어모든 full 화장품 dataset:
     # documents = [Document(page_content=text, metadata={"source": url})] # 제품명 : 설화수, 가격 : 4,000원 이런 식으로 임베딩 하지 않은 애들을 넣는다 

    
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=200
    )
    splits = text_splitter.split_documents(documents)
    
    return splits

In [72]:
def create_vectorstore(splits): 
    vectorstore = Chroma.from_documents(documents=splits, embedding=hf_embeddings)
    return vectorstore

In [73]:
topic = "흑백요리사"
# Load wikipedia documents for this topic
splits = load_Korean_wiki_docs(topic) 
# Create vectorstore with this fetched docs
vectorstore = create_vectorstore(splits)

In [77]:
splits

[Document(metadata={'source': 'https://ko.wikipedia.org/wiki/흑백요리사'}, page_content='《흑백요리사: 요리 계급 전쟁》(영어: Culinary Class Wars)은 넷플릭스의 요리 서바이벌 프로그램이다. 방송 직후 세계 여러 나라에서 시청률 1위를 기록했고, 대만인들의 한국 관광 열풍과 한국 음식에 대한 사랑을 불러일으켰다. 유명 레스토랑 셰프 등 100인의 요리사가 출연한다. 심사위원은 백종원과 안성재가 맡았다. 가제는 《무명요리사》였다.[1]')]

In [74]:
def create_rag_chain(vectorstore):
    prompt_template = """문맥을 참고하여 질문에 정확하고 간결하게 답하십시오.
    문맥: {context}
    질문: {question}
    답변:"""
    
    PROMPT = PromptTemplate(
        template=prompt_template, input_variables=["context", "question"]
    )

    chain_type_kwargs = {"prompt": PROMPT}

    # Make context shorter
    # def short_context(context, max_length=300):
    #     return context[:max_length] if len(context) > max_length else context
    
    # class ShortContextRetriever(BaseRetriever):
    #     def __init__(self, retriever):
    #         super().__init__()
    #         self._retriever = retriever
        
    #     def get_relevant_documents(self, query):
    #         docs = self._retriever.get_relevant_documents(query)
    #         for doc in docs:
    #             doc.page_content = short_context(doc.page_content)
    #         return docs
    
    # retriever = ShortContextRetriever(vectorstore.as_retriever())

    qa_chain = RetrievalQA.from_chain_type(
        llm=hf_llm,
        chain_type="stuff",
        retriever=vectorstore.as_retriever(),
        chain_type_kwargs=chain_type_kwargs,
        return_source_documents=True
    )
    
    return qa_chain

In [75]:
# create langchang RAG chain
qa_chain = create_rag_chain(vectorstore)

In [41]:
docs = vectorstore.as_retriever().get_relevant_documents(question)
docs

<ipython-input-41-ca9a4e50d709>:1: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  docs = vectorstore.as_retriever().get_relevant_documents(question)


[Document(metadata={'source': 'https://ko.wikipedia.org/wiki/흑백요리사'}, page_content='《흑백요리사: 요리 계급 전쟁》(영어: Culinary Class Wars)은 넷플릭스의 요리 서바이벌 프로그램이다. 방송 직후 세계 여러 나라에서 시청률 1위를 기록했고, 대만인들의 한국 관광 열풍과 한국 음식에 대한 사랑을 불러일으켰다. 유명 레스토랑 셰프 등 100인의 요리사가 출연한다. 심사위원은 백종원과 안성재가 맡았다. 가제는 《무명요리사》였다.[1]'),
 Document(metadata={'source': 'https://ko.wikipedia.org/wiki/흑백요리사'}, page_content='《흑백요리사: 요리 계급 전쟁》(영어: Culinary Class Wars)은 넷플릭스의 요리 서바이벌 프로그램이다. 방송 직후 세계 여러 나라에서 시청률 1위를 기록했고, 대만인들의 한국 관광 열풍과 한국 음식에 대한 사랑을 불러일으켰다. 유명 레스토랑 셰프 등 100인의 요리사가 출연한다. 심사위원은 백종원과 안성재가 맡았다. 가제는 《무명요리사》였다.[1]'),
 Document(metadata={'source': 'https://ko.wikipedia.org/wiki/흑백요리사'}, page_content='《흑백요리사: 요리 계급 전쟁》(영어: Culinary Class Wars)은 넷플릭스의 요리 서바이벌 프로그램이다. 방송 직후 세계 여러 나라에서 시청률 1위를 기록했고, 대만인들의 한국 관광 열풍과 한국 음식에 대한 사랑을 불러일으켰다. 유명 레스토랑 셰프 등 100인의 요리사가 출연한다. 심사위원은 백종원과 안성재가 맡았다. 가제는 《무명요리사》였다.[1]')]

In [42]:
docs = vectorstore.similarity_search(question, k=4)
docs

[Document(metadata={'source': 'https://ko.wikipedia.org/wiki/흑백요리사'}, page_content='《흑백요리사: 요리 계급 전쟁》(영어: Culinary Class Wars)은 넷플릭스의 요리 서바이벌 프로그램이다. 방송 직후 세계 여러 나라에서 시청률 1위를 기록했고, 대만인들의 한국 관광 열풍과 한국 음식에 대한 사랑을 불러일으켰다. 유명 레스토랑 셰프 등 100인의 요리사가 출연한다. 심사위원은 백종원과 안성재가 맡았다. 가제는 《무명요리사》였다.[1]'),
 Document(metadata={'source': 'https://ko.wikipedia.org/wiki/흑백요리사'}, page_content='《흑백요리사: 요리 계급 전쟁》(영어: Culinary Class Wars)은 넷플릭스의 요리 서바이벌 프로그램이다. 방송 직후 세계 여러 나라에서 시청률 1위를 기록했고, 대만인들의 한국 관광 열풍과 한국 음식에 대한 사랑을 불러일으켰다. 유명 레스토랑 셰프 등 100인의 요리사가 출연한다. 심사위원은 백종원과 안성재가 맡았다. 가제는 《무명요리사》였다.[1]'),
 Document(metadata={'source': 'https://ko.wikipedia.org/wiki/흑백요리사'}, page_content='《흑백요리사: 요리 계급 전쟁》(영어: Culinary Class Wars)은 넷플릭스의 요리 서바이벌 프로그램이다. 방송 직후 세계 여러 나라에서 시청률 1위를 기록했고, 대만인들의 한국 관광 열풍과 한국 음식에 대한 사랑을 불러일으켰다. 유명 레스토랑 셰프 등 100인의 요리사가 출연한다. 심사위원은 백종원과 안성재가 맡았다. 가제는 《무명요리사》였다.[1]')]

In [82]:
# It seems vectorDB loading from embedding model works fine, but seems llm model does not.
# Some Korean llm model seems to work fine in text-generation task, but for Question-Ansering task, we might need another approach.

In [7]:
## 2. Use QA pipeline with vectorstor similarity search¶

In [83]:
# import torch
from transformers import AutoModelForQuestionAnswering, AutoTokenizer, pipeline

# Load model and tokenizer
model_name = "yjgwak/klue-bert-base-finetuned-squard-kor-v1"
model = AutoModelForQuestionAnswering.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Set Q_A pipeline
qa_pipeline = pipeline("question-answering", model=model, tokenizer=tokenizer)

config.json:   0%|          | 0.00/635 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/367 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/246k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

Device set to use cpu


In [84]:
# Example: define question and context 
question = "오늘 날씨 어때?"
context = "오늘의 날씨는 맑고 따뜻한 기온이 유지될 것으로 보입니다."

# model chain
result = qa_pipeline(question=question, context=context)

# Result
print("질문:", question)
print("답변:", result['answer'])

질문: 오늘 날씨 어때?
답변: 맑고 따뜻한 기온이


In [85]:
# search context in VectorStore
def retrieve_context(question, vectorstore):
    docs = vectorstore.similarity_search(question, k=4)
    if docs:
        return " ".join([doc.page_content for doc in docs])
        # return docs[0].page_content  # return first relevant doc
    else:
        return None

# Generate answer based on query and searched context similar to RAG chain
def answer_question_with_context(question, vectorstore):
    context = retrieve_context(question, vectorstore)
    if context:
        result = qa_pipeline(question=question, context=context)
        return result['answer'], context  # return answer and used source doc
    else:
        return "관련 문맥을 찾지 못했습니다.", None

In [86]:
# Example
question = "심사위원을 누가 맡았어?"

answer, used_context = answer_question_with_context(question, vectorstore)

print("질문:", question)
print("답변:", answer)
print("사용된 문맥:", used_context)

질문: 심사위원을 누가 맡았어?
답변: 백종원과 안성재가
사용된 문맥: 《흑백요리사: 요리 계급 전쟁》(영어: Culinary Class Wars)은 넷플릭스의 요리 서바이벌 프로그램이다. 방송 직후 세계 여러 나라에서 시청률 1위를 기록했고, 대만인들의 한국 관광 열풍과 한국 음식에 대한 사랑을 불러일으켰다. 유명 레스토랑 셰프 등 100인의 요리사가 출연한다. 심사위원은 백종원과 안성재가 맡았다. 가제는 《무명요리사》였다.[1] 《흑백요리사: 요리 계급 전쟁》(영어: Culinary Class Wars)은 넷플릭스의 요리 서바이벌 프로그램이다. 방송 직후 세계 여러 나라에서 시청률 1위를 기록했고, 대만인들의 한국 관광 열풍과 한국 음식에 대한 사랑을 불러일으켰다. 유명 레스토랑 셰프 등 100인의 요리사가 출연한다. 심사위원은 백종원과 안성재가 맡았다. 가제는 《무명요리사》였다.[1] 《흑백요리사: 요리 계급 전쟁》(영어: Culinary Class Wars)은 넷플릭스의 요리 서바이벌 프로그램이다. 방송 직후 세계 여러 나라에서 시청률 1위를 기록했고, 대만인들의 한국 관광 열풍과 한국 음식에 대한 사랑을 불러일으켰다. 유명 레스토랑 셰프 등 100인의 요리사가 출연한다. 심사위원은 백종원과 안성재가 맡았다. 가제는 《무명요리사》였다.[1] 《흑백요리사: 요리 계급 전쟁》(영어: Culinary Class Wars)은 넷플릭스의 요리 서바이벌 프로그램이다. 방송 직후 세계 여러 나라에서 시청률 1위를 기록했고, 대만인들의 한국 관광 열풍과 한국 음식에 대한 사랑을 불러일으켰다. 유명 레스토랑 셰프 등 100인의 요리사가 출연한다. 심사위원은 백종원과 안성재가 맡았다. 가제는 《무명요리사》였다.[1]


In [87]:
# It seems the best and simple and cost-free option when OpenAI api cannot be used.

In [9]:
## 3. Use Gemini+RAG

In [71]:
pip install -q langchain langchain-community langchain_huggingface chromadb google-generativeai

Note: you may need to restart the kernel to use updated packages.


In [72]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.schema import Document
from langchain.llms import OpenAI
import google.generativeai as genai
import os

In [74]:
# os.environ["HUGGINGFACEHUB_API_TOKEN"] = "YOUR-API-KEY"
genai_api_key = "YOUR-API-KEY"

In [76]:
genai.configure(api_key=genai_api_key)

In [75]:
pip install genai

Note: you may need to restart the kernel to use updated packages.


In [77]:
# 1. Gemini model
gemini_model = genai.GenerativeModel('gemini-1.5-flash')

# 2. embedding model
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")

<ipython-input-77-5839f8a42bf5>:5: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")


In [93]:
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.schema import Document

# Sample documents
docs = [
    Document(page_content="한국어 챗봇은 자연어 처리 기술을 사용하여 사용자와 대화를 나눕니다.", metadata={"source": "doc1"}),
    Document(page_content="인공지능을 활용한 챗봇은 여러 산업에서 사용되고 있습니다.", metadata={"source": "doc2"}),
    Document(page_content="한국어와 영어를 동시에 지원하는 챗봇이 점점 늘어나고 있습니다.", metadata={"source": "doc3"}),
    Document(page_content="챗봇은 고객 서비스를 개선하고 사용자 경험을 향상시키는 데 중요한 역할을 합니다.", metadata={"source": "doc4"})
]

# 텍스트 분할기 생성
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)

# 문서를 분할
splits = text_splitter.split_documents(docs)

# Embedding 모델 설정 (이미 잘 설정되어 있다고 가정)
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")

# Chroma DB 생성 (새로운 디렉토리에 저장)
persist_directory = "./new_chroma_db"
vectorstore = Chroma.from_documents(docs, embedding=embedding_model, persist_directory=persist_directory)

# 정상적으로 vectorstore 생성되었음을 출력
print("Vectorstore created successfully!")



Vectorstore created successfully!


In [99]:
# RAG using prompt
def rag_chatbot(question):
    context_doc = vectorstore.similarity_search(question, k=1)
   # context = context_doc[0].page_content if context_doc else "정보를 찾을 수 없습니다."

    context = " ".join([doc.page_content for doc in context_doc])

    prompt = f"Context: {context}\nQuestion: {question}\n한문장으로 대답해줘:"
    # response = gemini_model(prompt)
    
    response = gemini_model.generate_content(prompt)
    answer = response.candidates[0].content.parts[0].text

    # print("출처 문서:", context)
    return answer

In [103]:
# sample question
question = "영등포 근처에 IT 관련 배울 수 있는 학원이 있니? 있으면 구체적으로 이름 좀 알려줘"
response = rag_chatbot(question)

print("질문:", question)
print("답변:", response)

질문: 영등포 근처에 IT 관련 배울 수 있는 학원이 있니? 있으면 구체적으로 이름 좀 알려줘
답변: 네, 영등포 근처에는 코딩 관련 학원인 [학원 이름 1], [학원 이름 2] 등이 있으며,  자세한 정보는 각 학원 웹사이트에서 확인 가능합니다.

